# Real world data Sales Analysis

In [1]:
import pandas as pd

In [2]:
dataset_info = [
        # hidden
]

datasets = {'b2c':{}, 'b2b':{}}

for filename, category in dataset_info:
    dataset = pd.read_csv(filename)
    datasets[category][filename] = dataset
    
merged_dataset_b2b = pd.concat(datasets['b2b'].values(), ignore_index = True)
merged_dataset_b2c = pd.concat(datasets['b2c'].values(), ignore_index = True)

merged_dataset = pd.concat([merged_dataset_b2b,merged_dataset_b2c], ignore_index =True)
merged_dataset.columns

Index(['Seller Gstin', 'Invoice Number', 'Invoice Date', 'Transaction Type',
       'Order Id', 'Shipment Id', 'Shipment Date', 'Order Date',
       'Shipment Item Id', 'Quantity', 'Item Description', 'Asin', 'Hsn/sac',
       'Sku', 'Product Tax Code', 'Bill From City', 'Bill From State',
       'Bill From Country', 'Bill From Postal Code', 'Ship From City',
       'Ship From State', 'Ship From Country', 'Ship From Postal Code',
       'Ship To City', 'Ship To State', 'Ship To Country',
       'Ship To Postal Code', 'Invoice Amount', 'Tax Exclusive Gross',
       'Total Tax Amount', 'Cgst Rate', 'Sgst Rate', 'Utgst Rate', 'Igst Rate',
       'Compensatory Cess Rate', 'Principal Amount', 'Principal Amount Basis',
       'Cgst Tax', 'Sgst Tax', 'Utgst Tax', 'Igst Tax',
       'Compensatory Cess Tax', 'Shipping Amount', 'Shipping Amount Basis',
       'Shipping Cgst Tax', 'Shipping Sgst Tax', 'Shipping Utgst Tax',
       'Shipping Igst Tax', 'Shipping Cess Tax', 'Gift Wrap Amount',
     

#### To protect privacy of the data

In [3]:
unique_product_names = merged_dataset['Item Description'].unique()

In [4]:
product_mapping = { product_name:f"Product {index+1}" for index, product_name in enumerate (unique_product_names)}

merged_dataset['Product Name'] = merged_dataset['Item Description'].replace(product_mapping)


## Filtering Data

In [5]:
merged_dataset.dropna(how= 'all', inplace = True)

In [6]:
merged_dataset.query("`Transaction Type` != 'Cancel'", inplace = True)
# Alternate

# merged_dataset.where(merged_dataset['Transaction Type'] != 'Cancel', inplace = True)


In [7]:
merged_dataset.drop(['Seller Gstin','Order Id','Invoice Number','Invoice Date','Shipment Id', 'Shipment Date','Shipment Item Id','Product Tax Code', 'Bill From City', 'Bill From State',
       'Bill From Country', 'Bill From Postal Code','Tax Exclusive Gross',
       'Total Tax Amount', 'Cgst Rate', 'Sgst Rate', 'Utgst Rate', 'Igst Rate',
       'Compensatory Cess Rate', 'Principal Amount', 'Principal Amount Basis',
       'Cgst Tax', 'Sgst Tax', 'Utgst Tax', 'Igst Tax',
       'Compensatory Cess Tax', 'Shipping Amount', 'Shipping Amount Basis',
       'Shipping Cgst Tax', 'Shipping Sgst Tax', 'Shipping Utgst Tax',
       'Shipping Igst Tax', 'Shipping Cess Tax', 'Gift Wrap Amount',
       'Gift Wrap Amount Basis', 'Gift Wrap Cgst Tax', 'Gift Wrap Sgst Tax',
       'Gift Wrap Utgst Tax', 'Gift Wrap Igst Tax',
       'Gift Wrap Compensatory Cess Tax', 'Item Promo Discount Basis', 'Item Promo Tax',
       'Shipping Promo Discount', 'Shipping Promo Discount Basis',
       'Shipping Promo Tax', 'Gift Wrap Promo Discount',
       'Gift Wrap Promo Discount Basis', 'Gift Wrap Promo Tax',
       'Tcs Cgst Rate', 'Tcs Cgst Amount', 'Tcs Sgst Rate', 'Tcs Sgst Amount',
       'Tcs Utgst Rate', 'Tcs Utgst Amount', 'Tcs Igst Rate',
       'Tcs Igst Amount', 'Warehouse Id','Payment Method Code', 'Bill To City', 'Bill To State',
       'Bill To Country', 'Bill To Postalcode', 'Customer Bill To Gstid',
       'Customer Ship To Gstid', 'Buyer Name', 'Credit Note No',
       'Credit Note Date', 'Irn Number', 'Irn Filing Status', 'Irn Date',
       'Irn Error Code', 'Shipping Cess Tax Amount','Asin','Hsn/sac','Sku' ],axis =1,  inplace = True)

## Creating Custom Dataset / Pre Processing 

In [8]:
c_dataset = merged_dataset[[ 'Transaction Type',
       'Quantity', 'Product Name']].copy()

In [9]:
c_dataset['Order Date'] = pd.to_datetime(merged_dataset['Order Date'])
c_dataset['Fulfillment Channel'] = merged_dataset['Fulfillment Channel']

## Creating a column with statenames from the ship to postal code.
####   1. We are gonna use a dataframe that has most of the postal codes with their statenames.
####   2. For the Postal code which are not present will use another method to get the state names.

In [10]:
# lets First create a dictionary to map the postal codes to statenames
pincode_lookup_df = pd.read_csv('Pincode_lookup.csv')

#creating a dict with key as Pincode and value as Statename
pincode_lookup_dict = pincode_lookup_df.set_index('Pincode')['StateName'].to_dict()

In [11]:
#mapping state postal code to the pincode_lookup_dict

c_dataset['Shipping State'] = merged_dataset['Ship To Postal Code'].map(pincode_lookup_dict)

In [12]:
c_dataset['Shipping State'].isnull().sum()

87

--> There are still 87 rows with unmatched statename to solve this

--> The first two numbers of the postal codes have direct relation with the state name ex: pincodes starting with 11 are of delhi, 
12 is for haryana we can use this to fill the rest of the rows with right state name


## We are gonna use another dataframe that is pincode_lookup_or which as state pincode relation with the first two numbers of the postal code

In [13]:
pincode_lookup_or = pd.read_csv('State OR lookup.csv')

#creating a dict to map
pincode_lookup_or['First_Numbers'] = pincode_lookup_or['First_Numbers'].astype(str)
pincode_lookup_or_dict = pincode_lookup_or.set_index('First_Numbers')['State_Name'].to_dict()


# mapping the dict and creating new column with the relative state name and then fill the nulls of shiiping state with these new state names

c_dataset['relative statename'] = merged_dataset['Ship To Postal Code'].astype(str).str[:2].map(pincode_lookup_or_dict)

c_dataset['Shipping State'] = c_dataset['Shipping State'].fillna(c_dataset['relative statename'])

c_dataset.drop('relative statename', axis=1, inplace=True)
c_dataset['Shipping State'].isnull().sum()

0

## Seggregating Refunds/Returns, Promotional Sales, Organic Sales and Gross Sales

#### Refunds/Retruns

In [14]:
c_dataset['Refunds/Returns'] = (merged_dataset['Invoice Amount'].where(merged_dataset['Transaction Type'] == 'Refund')).fillna(0)

#alternate 1

# c_dataset['Refunds/Returns'] = merged_dataset.query("`Transaction Type` == 'Refund'")['Invoice Amount']
# c_dataset['Refunds/Returns'] = c_dataset['Refunds/Returns'].fillna(0)

#alternate 2 

# c_dataset['Refunds/Returns'] = merged_dataset['Invoice Amount'][merged_dataset['Transaction Type'] == 'Refund']
# c_dataset['Refunds/Returns'] = c_dataset['Refunds/Returns'].fillna(0)



#### Promotional Sales

In [15]:
c_dataset['Promotional Sales'] = (merged_dataset['Invoice Amount'].where(merged_dataset['Item Promo Discount']<0)).fillna(0)

#alternate 1
# c_dataset['Promotional Sales'] = merged_dataset.query("`Item Promo Discount` < 0")['Invoice Amount']
# c_dataset['Promotional Sales'] = c_dataset['Promotional Sales'].fillna(0) 

#alternate 2
# c_dataset['Promotional Sales'] = merged_dataset['Invoice Amount'][merged_dataset['Item Promo Discount']<0]
# c_dataset['Promotional Sales'] = c_dataset['Promotional Sales'].fillna(0) 


#### Organice Sales

In [16]:
c_dataset['Organice Sales'] = (merged_dataset['Invoice Amount'].where((merged_dataset['Transaction Type']!='Refund') & (merged_dataset['Item Promo Discount']==0))).fillna(0)

In [17]:
c_dataset['Gross Sales'] = merged_dataset['Invoice Amount'].abs()

## Processed Data ready for insights

In [19]:
c_dataset.to_csv('Optmised Analysis Data.csv', index=False)
c_dataset

,Transaction Type,Quantity,Product Name,Order Date,Fulfillment Channel,Shipping State,Refunds/Returns,Promotional Sales,Organice Sales,Gross Sales
1,Shipment,2,Product 2,2022-02-13 23:29:04,MFN,Madhya Pradesh,0.0,0.0,1198.0,1198.0
2,Shipment,1,Product 2,2022-02-14 00:05:14,MFN,Maharashtra,0.0,539.1,0.0,539.1
4,Shipment,1,Product 3,2022-02-18 17:27:15,MFN,Uttar Pradesh,0.0,0.0,699.0,699.0
5,Shipment,1,Product 4,2022-02-23 11:28:24,AFN,Maharashtra,0.0,449.1,0.0,449.1
7,Shipment,2,Product 5,2022-02-25 20:33:16,MFN,Telangana,0.0,0.0,3998.0,3998.0
...,...,...,...,...,...,...,...,...,...,...
8296,Shipment,1,Product 24,2023-05-31 14:18:41,AFN,Telangana,0.0,0.0,499.0,499.0
8297,Shipment,1,Product 7,2023-05-31 09:57:04,AFN,Uttar Pradesh,0.0,0.0,599.0,599.0
8298,Shipment,1,Product 24,2023-05-28 13:10:13,AFN,Karnataka,0.0,0.0,499.0,499.0
8299,Shipment,1,Product 7,2023-05-31 19:45:44,AFN,Maharashtra,0.0,0.0,599.0,599.0


# Visualisation 

[Power BI Visualisation](https://app.powerbi.com/view?r=eyJrIjoiN2E5MDkzNDItMDA1OS00MTM0LWE5NGQtODFhNGQ4MDk0YzZmIiwidCI6ImJmNDE4ZmE0LWM3NzQtNDViMS05YWZiLTM0NjgyNGVlYWZlMSIsImMiOjEwfQ%3D%3D&pageName=ReportSection)
